# BX Books dataset  - Data Cleaning
  
Source: The dataset is available [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from scipy.sparse import csr_matrix
import numpy as np

%matplotlib inline

In [2]:
for line in open('/Users/janakajain/GitHub/E4571-Personalisation-Theory-Project/raw-data/users.csv','r',encoding = 'ISO-8859-1').readlines()[:10]:
    a = re.compile(pattern = '^"[0-9];\"\"[a-z", ]+;[A-Z]+\"[,]+')
    _id_regex = re.compile(pattern = '^"[0-9];')
    _id = _id_regex.match(line)
    print(line)

"User-ID;""Location"";""Age""",,,,,,,,

"1;""nyc", new york," usa"";NULL",,,,,,

"2;""stockton", california," usa"";""18""",,,,,,

"3;""moscow", yukon territory," russia"";NULL",,,,,,

"4;""porto", v.n.gaia," portugal"";""17""",,,,,,

"5;""farnborough", hants," united kingdom"";NULL",,,,,,

"6;""santa monica", california," usa"";""61""",,,,,,

"7;""washington", dc," usa"";NULL",,,,,,

"8;""timmins", ontario," canada"";NULL",,,,,,

"9;""germantown", tennessee," usa"";NULL",,,,,,



### Ratings dataset

In [3]:
data = pd.read_csv('/Users/janakajain/GitHub/E4571-Personalisation-Theory-Project/raw-data/ratings.csv', 
                   encoding='ISO-8859-1', 
                   sep = r'\";\"')

/Users/janakajain/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [4]:
data.head(10)

,"""User-ID",ISBN,"Book-Rating"""
0,"""276725",034545104X,"0"""
1,"""276726",0155061224,"5"""
2,"""276727",0446520802,"0"""
3,"""276729",052165615X,"3"""
4,"""276729",0521795028,"6"""
5,"""276733",2080674722,"0"""
6,"""276736",3257224281,"8"""
7,"""276737",0600570967,"6"""
8,"""276744",038550120X,"7"""
9,"""276745",342310538,"10"""


The ratings dataset contains inconsistencies which cannot be cleaned effectively using regex. This has been addressed through manual cleaning in the following steps. The resulting data has been stored in cleaned CSV and pickle files.

In [5]:
data['user_id'] = data['"User-ID'].str.replace('"','').astype(int)

In [6]:
data['book_rating'] = data['Book-Rating"'].str.replace('"','').astype(int)

In [7]:
data['isbn'] = data['ISBN']

In [8]:
df = data[['user_id','isbn','book_rating']]

In [9]:
# df.to_csv('clean-data/ratings.csv')

In [10]:
# df.to_pickle('clean-data/ratings.pickle')

---  
### Books Dataset

In [11]:
books = pd.read_csv('/Users/janakajain/GitHub/E4571-Personalisation-Theory-Project/raw-data/books.csv', 
                    sep = r'\";\"'
                   )

/Users/janakajain/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [12]:
books['isbn'] = books['"ISBN'].str.replace('"','')

In [13]:
books['image-url-l'] = books['Image-URL-L"'].str.replace('"','')

In [14]:
del books['"ISBN']
del books['Image-URL-L"']

In [15]:
# books.to_csv('clean-data/books.csv')
# books.to_pickle('clean-data/books.pickle')

---  
### Users Dataset

In [16]:
# users = pd.read_csv('/Users/janakajain/GitHub/E4571-Personalisation-Theory-Project/raw-data/users.csv', 
#                     sep = r'\";\"'
#                    )

In [17]:
# df2 = df.pivot_table(index = 'user_id', columns = 'isbn', values = 'book_rating')

### Saving the ratings as user-item matrix

In [26]:
user_unique = list(sorted(df.user_id.unique()))
isbn_unique = list(sorted(df.isbn.unique()))

In [27]:
data = df['book_rating'].tolist()
row = df.user_id.astype('category', categories = user_unique).cat.codes
col = df.isbn.astype('category', categories = isbn_unique).cat.codes

In [30]:
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(user_unique), len(isbn_unique)))

In [54]:
np.save(arr=sparse_matrix,file='matrix')